In [ ]:
import time
from google.colab import drive
import os
import requests
import json
from PIL import Image
from io import BytesIO
from tqdm import tqdm

# Google Driveをマウント
drive.mount('/content/drive')

# 保存先ディレクトリを設定
save_dir = "/content/drive/MyDrive/Safebooru_Images/"
metadata_file = os.path.join(save_dir, "metadata.json")  # メタデータ保存ファイル
os.makedirs(save_dir, exist_ok=True)

# Safebooru APIからデータを取得
def fetch_safebooru_images(tags="", limit=100, page=1):
    """
    Safebooru APIから画像情報を取得
    :param tags: 取得するタグ
    :param limit: 1リクエストあたりの画像数 (最大100)
    :param page: ページ番号
    :return: 画像情報のリスト
    """
    base_url = "https://safebooru.donmai.us/posts.json"
    params = {
        "tags": tags,
        "limit": limit,
        "page": page
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

# 画像をダウンロードして高さ1024pxにリサイズ、JPEG形式で保存
def download_image_as_jpeg(url, save_path, target_height=1024):
    """
    画像を指定したURLからダウンロードし、高さ1024pxにリサイズ、JPEG形式に変換して保存
    :param url: 画像URL
    :param save_path: 保存先のファイルパス
    :param target_height: リサイズ後の高さ
    """
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            # アスペクト比を保持したまま高さを1024pxにリサイズ
            width, height = image.size
            target_width = int((target_height / height) * width)  # 新しい幅を計算
            image_resized = image.resize((target_width, target_height), Image.LANCZOS)

            # JPEG形式で保存
            jpeg_path = os.path.splitext(save_path)[0] + ".jpg"
            image_resized.convert("RGB").save(jpeg_path, "JPEG")
        else:
            print(f"Failed to download {url}")
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# 著作者情報を含む画像メタデータを保存
def save_metadata(metadata, file_path):
    """
    メタデータをJSONファイルに保存
    :param metadata: 画像のメタデータリスト
    :param file_path: 保存先のファイルパス
    """
    with open(file_path, "w") as f:
        json.dump(metadata, f, indent=4, ensure_ascii=False)

# 画像を一括ダウンロード
def download_safebooru_images(tags="", max_images=500, delay=3):
    """
    Safebooruから画像を一括取得してGoogle Driveに保存
    :param tags: 取得する画像のタグ
    :param max_images: 最大取得枚数
    :param delay: 各リクエストの待機時間 (秒)
    """
    metadata = []  # メタデータリスト
    downloaded_count = 0
    page = 1

    # tqdmを使用したプログレスバー
    with tqdm(total=max_images, desc="Downloading images") as pbar:
        while downloaded_count < max_images:
            images = fetch_safebooru_images(tags=tags, limit=100, page=page)
            if not images:
                print("No more images to fetch.")
                break
            for img in images:
                if downloaded_count >= max_images:
                    break
                file_url = img.get("file_url")
                if file_url:
                    file_name = os.path.basename(file_url)
                    save_path = os.path.join(save_dir, file_name)

                    # 画像のダウンロードとリサイズ、JPEG変換
                    download_image_as_jpeg(file_url, save_path)

                    # 著作者情報とメタデータを取得
                    metadata.append({
                        "id": img.get("id"),
                        "file_name": os.path.splitext(file_name)[0] + ".jpg",  # 拡張子をJPEGに
                        "artist": img.get("tag_string_artist", "Unknown"),
                        "tags": img.get("tag_string"),
                        "source": img.get("source", "N/A"),
                        "url": f"https://safebooru.donmai.us/posts/{img.get('id')}"
                    })

                    downloaded_count += 1
                    pbar.update(1)  # プログレスバーを更新
                    time.sleep(delay)  # 指定秒数待機
            page += 1

    # メタデータを保存
    save_metadata(metadata, metadata_file)
    print(f"\nDownloaded {downloaded_count} images.")
    print(f"Metadata saved to {metadata_file}")

# タグを指定して画像を取得
download_safebooru_images(tags="scenery", max_images=100, delay=1)  # 各画像間5秒間隔


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Error downloading https://cdn.donmai.us/original/ff/f1/fff1d190332f7abc6c65dc409ea5d6f4.mp4: cannot identify image file <_io.BytesIO object at 0x78ac30fa0db0>


Error downloading https://cdn.donmai.us/original/27/f0/27f0065da8fd95b405e1dd1a5b8a65b8.mp4: cannot identify image file <_io.BytesIO object at 0x78ac41244680>



Downloaded 100 images.
Metadata saved to /content/drive/MyDrive/Safebooru_Images/metadata.json
